In [113]:
CLIENT_ID = 'c6a1856985ec4573ac83a02c93b1e77d'
CLIENT_SECRET = 'c592ffedc2164b60ac858caf452f5afa'
AUTH_URL = 'https://accounts.spotify.com/api/token'

import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def spotDf(spotify_response, root_key, normalize_all):
    df = pd.DataFrame()
    data = spotify_response.get(root_key, [])
    if(normalize_all==True):
        for item in data:
            df = df._append(pd.json_normalize(item))
    else: df = df._append(pd.json_normalize(data))
    for column in df.columns:
         df[column] = df[column].apply(
                lambda x: ', '.join([spotify_response['name'] for spotify_response in x]) if isinstance(x, list) and all(isinstance(spotify_response, dict) for spotify_response in x) else x
                )
         return df

In [ ]:
artista = []
musica_titulo = []
musica_popularidade = []
musica_duracao = []
artista_id = []
musica_id = []
musica_data_lancamento = []
    
for i in range(0, 1000, 50):
    resultado = sp.search(q=f'genre:K-pop year:2022', type='track', limit=50, offset=i, market = 'BR')
    for i, t in enumerate(resultado['tracks']['items']):
        artista.append(t['artists'][0]['name'])
        artista_id.append(t['artists'][0]['id'])
        musica_titulo.append(t['name'])
        musica_id.append(t['id'])
        musica_popularidade.append(t['popularity'])
        musica_duracao.append(t['duration_ms'])
        musica_data_lancamento.append(t['album']['release_date'])

musica_df = pd.DataFrame({'artista': artista,
                               'musica_titulo': musica_titulo,
                               'musica_popularidade': musica_popularidade,
                               'musica_duracao': musica_duracao,
                               'artista_id': artista_id,
                               'musica_id': musica_id,
                               'musica_data_lancamento': musica_data_lancamento
                               })

# Set options to display all columns and avoid truncation
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', None)         # Auto-adjust width to fit the screen
pd.set_option('display.colheader_justify', 'left')
print(musica_df)

In [ ]:
artista_nome = []
artista_id = []
artista_popularidade = []
artista_seguidores = []
artista_genero = []

for i in range(0, 20, 50):
    artista_resultados = sp.search(q=f'genre:k-pop, year:2023', type='artist', limit=50, offset=i)
    for artist in artista_resultados['artists']['items']:
        artista_nome.append(artist['name'])
        artista_id.append(artist['id'])
        artista_popularidade.append(artist['popularity'])
        artista_genero.append(artist['genres'])
        artista_seguidores.append(artist['followers'])

artista_df = pd.DataFrame({'artista_nome': artista_nome,
                               'artista_id': artista_id,
                               'artista_popularidade': artista_popularidade,
                               'artista_seguidores': artista_seguidores,
                               'artista_genero': artista_genero
                               })

print(artista_df)

In [ ]:
musica_features = []
for m_id in musica_df['musica_id']:
    af = sp.audio_features(m_id)
    musica_features.append(af)

mf_df = pd.DataFrame(columns=['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'id'])

i = 0
for item in musica_features:
    for feat in item:
        mf_df = mf_df._append({'danceability': feat['danceability'],
                              'energy': feat['energy'],
                              'loudness': feat['loudness'],
                              'speechiness': feat['speechiness'],
                              'acousticness': feat['acousticness'],
                              'instrumentalness': feat['instrumentalness'],
                              'liveness': feat['liveness'],
                              'valence': feat['valence'],
                              'tempo': feat['tempo'],
                              'duration_ms': feat['duration_ms'],
                              'time_signature': feat['time_signature'],
                              'musica_titulo': musica_titulo[i]}, ignore_index=True)
        i = i+1
#elimina colunas iguais
mf_df.drop('id', axis = 1, inplace = True)
print(mf_df)

In [ ]:
columns = ['artists', 'duration_ms', 'id',  'name',
       'popularity', 'album.name', 'album.release_date']
artists = sp.artist_top_tracks('66CXWjxzNUsdJxJ2JdwvnR', country='US')
df = spotDf(artists, 'tracks', True)
filtered_df = df[columns]
print(filtered_df)